# Comprensión y EDA (Exploratory Data Analysis)

Este notebook contiene el Análisis Exploratorio de Datos (EDA) para el proyecto de riesgo crediticio.
Se incluyen análisis univariables, bivariables y multivariables, así como limpieza y caracterización de datos.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Configuración de visualización
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Cargar datos
data_path = '../../Base_de_datos.xlsx'
if os.path.exists(data_path):
    df = pd.read_excel(data_path)
    print(f"Datos cargados: {df.shape}")
else:
    print(f"No se encontró el archivo en {data_path}")

## 1. Exploración Inicial de Datos

### Descripción General y Caracterización
Identificación de tipos de variables: categóricas, numéricas, ordinales, nominales.

In [ ]:
df.info()

In [ ]:
# Revisión de nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

In [ ]:
# Unificar representación de Nulos (si aplica)
# Ejemplo: Reemplazar '?' o 'NA' string por np.nan si existieran
# df.replace('?', np.nan, inplace=True)
print("Revisión de valores únicos en columnas object para detectar nulos ocultos:")
for col in df.select_dtypes(include=['object']).columns:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# Conversión de tipos de datos
# Asegurar que las fechas sean datetime
if 'fecha_prestamo' in df.columns:
    df['fecha_prestamo'] = pd.to_datetime(df['fecha_prestamo'], errors='coerce')

# Asegurar que variables categóricas sean category o object
categorical_cols = ['tipo_laboral', 'nivel_educativo', 'sexo', 'estado_civil'] # Ajustar según columnas reales
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')

# Verificar tipos finales
print(df.dtypes)

In [ ]:
# Eliminación de variables irrelevantes
# Identificar columnas con valor único (sin varianza) o IDs que no aportan información predictiva
cols_to_drop = []
for col in df.columns:
    if df[col].nunique() == 1:
        cols_to_drop.append(col)
        
# ID columns (si existen y no son útiles)
# if 'id_cliente' in df.columns: cols_to_drop.append('id_cliente')

if cols_to_drop:
    print(f"Columnas a eliminar (sin varianza): {cols_to_drop}")
    df.drop(columns=cols_to_drop, inplace=True)
else:
    print("No se encontraron columnas con varianza cero para eliminar.")

## 2. Análisis Univariable
Estadísticas descriptivas y visualización de distribuciones.

In [ ]:
# Estadísticas descriptivas para numéricas
display(df.describe())

In [ ]:
# Estadísticas para categóricas
display(df.describe(include=['category', 'object']))

In [ ]:
# Visualización de variables numéricas
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
# Excluir target si es numérico pero binario
if 'Pago_atiempo' in num_cols: 
    num_cols = num_cols.drop('Pago_atiempo')

for col in num_cols:
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribución de {col}')
    
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot de {col}')
    plt.show()

In [ ]:
# Visualización de variables categóricas
cat_cols = df.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    plt.figure(figsize=(10, 5))
    sns.countplot(y=col, data=df, order=df[col].value_counts().index)
    plt.title(f'Distribución de {col}')
    plt.show()

In [ ]:
# Variable Objetivo
plt.figure(figsize=(6, 4))
sns.countplot(x='Pago_atiempo', data=df)
plt.title('Distribución de Variable Objetivo (Pago_atiempo)')
plt.show()
print(df['Pago_atiempo'].value_counts(normalize=True))

## 3. Análisis Bivariable
Relación con la variable objetivo.

In [ ]:
# Numéricas vs Objetivo
for col in num_cols:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='Pago_atiempo', y=col, data=df)
    plt.title(f'{col} vs Pago_atiempo')
    plt.show()

In [ ]:
# Categóricas vs Objetivo
for col in cat_cols:
    plt.figure(figsize=(10, 6))
    sns.countplot(y=col, hue='Pago_atiempo', data=df)
    plt.title(f'{col} vs Pago_atiempo')
    plt.legend(title='Pago a tiempo', loc='lower right')
    plt.show()

## 4. Análisis Multivariable
Correlaciones y relaciones complejas.

In [ ]:
# Matriz de Correlación
plt.figure(figsize=(12, 10))
corr_matrix = df.select_dtypes(include=['float64', 'int64']).corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
# Pairplot (muestra representativa)
selected_cols = ['capital_prestado', 'edad_cliente', 'puntaje', 'Pago_atiempo'] # Seleccionar las más relevantes
sns.pairplot(df[selected_cols].sample(min(1000, len(df))), hue='Pago_atiempo', diag_kind='kde')
plt.title('Pairplot de variables seleccionadas')
plt.show()